In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel
from pyspark.ml import Pipeline
from preprocessing.preprocessor import split_data, class_imbalance

%load_ext autoreload
%autoreload 2

In [2]:
spark = SparkSession.builder.appName("Pipeline").getOrCreate()

24/12/06 16:59:49 WARN Utils: Your hostname, Nikhils-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.206 instead (on interface en0)
24/12/06 16:59:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/06 16:59:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/06 16:59:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/06 16:59:49 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
# Load data
data = spark.read.csv("dataset/creditcard.csv", header=True, inferSchema=True).cache()
data.show(5)

24/12/06 16:59:53 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+------------------+-------------------+----------------+------------------+-------------------+-------------------+-------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+--------------------+-------------------+------------------+------------------+------------------+------------------+--------------------+-------------------+------+-----+
|Time|                V1|                 V2|              V3|                V4|                 V5|                 V6|                 V7|                V8|                V9|                V10|               V11|               V12|               V13|               V14|               V15|               V16|               V17|                V18|               V19|                V20|                 V21|                V22|     

In [4]:
# Preprocessing

# Create an assembler as the first stage of the pipeline
assembler = VectorAssembler(inputCols=data.columns[:-1], outputCol="features")

# Create a scaler as the second stage of the pipeline
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")

## Model Saving and Loading
Since we have already trained a model and saved it, we can load it instead of training a new model. However, we will check if the model exists in the models directory. If it does not exist, we will train a new model and save it.

In [5]:
# Check if we already have a model saved in models directory
# If not, create a new model
# If we have a model saved, load it
rf_model = None 
try:
    rf_model = RandomForestClassificationModel.load("models/credit_card_fraud_detection_model")
except Exception as e:
    print("Model not found. Training a new model.")
    rf_model = RandomForestClassifier(featuresCol="scaled_features", labelCol="Class") 

In [6]:
# Create a pipeline
pipeline = Pipeline(stages=[assembler, scaler, rf_model])

# Pipeline restriction
Now that we are inside the pipeline, the assembling and scaling will happen to the data we feed in

In [7]:
# Split training and testing data
df_balanced = class_imbalance(data)
train_split, test_split = split_data(df_balanced, 42)
train_unsplit, test_unsplit = split_data(data, 42)

Fraudulent transactions: 492
Non-fraudulent transactions: 284315 before balancing


In [8]:
print("Split data")
print(f"Train data count: {train_split.count()}")
print(f"Test data count: {test_split.count()}")

print("\nUnsplit data")
print(f"Train data count: {train_unsplit.count()}")
print(f"Test data count: {test_unsplit.count()}")

Split data
Train data count: 802
Test data count: 191

Unsplit data
Train data count: 228045
Test data count: 56762


In [9]:
# Fit the pipeline with unbalanced data
pipeline_model = pipeline.fit(train_unsplit)

In [15]:
# Make predictions
predictions = pipeline_model.transform(test_unsplit)
predictions.show(5)

+----+------------------+-----------------+------------------+------------------+------------------+------------------+--------------------+-------------------+------------------+-------------------+------------------+------------------+-------------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+------------------+-------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+------+-----+--------------------+--------------------+--------------------+--------------------+----------+
|Time|                V1|               V2|                V3|                V4|                V5|                V6|                  V7|                 V8|                V9|                V10|               V11|               V12|                V13|               V14|                V15|                V16|                V17|    

In [11]:
# Save the pipeline
# pipeline_model.write().overwrite().save("pipeline/credit_card_fraud_detection_pipeline")

In [4]:
predictions.printSchema()

NameError: name 'predictions' is not defined

In [3]:
# Evaluate the model

from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol="Class")
print(f"Area under ROC: {evaluator.evaluate(predictions)}")

AssertionError: 

In [2]:
# Plot ROC curve
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Convert the predictions to a Pandas DataFrame
predictions_df = predictions.select('Class', 'probability').toPandas()

# Get the false positive rate and true positive rate
y_true = predictions_df.select("Class").rdd.flatMap(lambda x: x).collect()
y_scores = predictions_df.select("probability").rdd.flatMap(lambda x: x[0][1]).collect()

fpr, tpr, _ = roc_curve(y_true, y_scores)

# Calculate the AUC
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=1, label=f'ROC curve (area = {roc_auc:.2f})')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

NameError: name 'predictions' is not defined